In [1]:
var openDigger = require('../src/open_digger');

In [2]:
var query = openDigger.driver.clickhouse.query;
query(    
    `SHOW DATABASES`
)
    .then(res=>{    
    console.log(res);
})

[
  { name: '_temporary_and_external_tables' },
  { name: 'default' },
  { name: 'github_log' },
  { name: 'system' }
]


In [9]:
var startYear = 2015, endYear = 2021; var startMonth = 1, endMonth = 12;
var years = [], months = [];
for (let i = startYear; i <= endYear; i++) {
    years[i - startYear] = i;
}
for (let i = startMonth; i <= endMonth; i++) {
    months[i - startMonth] = i;
}
var yms = []
for(let i = 0; i < years.length; i++) {
    for (let j = 0; j < months.length; j++) {
        yms.push({'y': years[i], 'm': months[j]});
    }
}
yms_axis = yms.map(item => item['y'].toString() + '-' + item['m'].toString())

[
  '2015-1',  '2015-2',  '2015-3',  '2015-4',  '2015-5',
  '2015-6',  '2015-7',  '2015-8',  '2015-9',  '2015-10',
  '2015-11', '2015-12', '2016-1',  '2016-2',  '2016-3',
  '2016-4',  '2016-5',  '2016-6',  '2016-7',  '2016-8',
  '2016-9',  '2016-10', '2016-11', '2016-12', '2017-1',
  '2017-2',  '2017-3',  '2017-4',  '2017-5',  '2017-6',
  '2017-7',  '2017-8',  '2017-9',  '2017-10', '2017-11',
  '2017-12', '2018-1',  '2018-2',  '2018-3',  '2018-4',
  '2018-5',  '2018-6',  '2018-7',  '2018-8',  '2018-9',
  '2018-10', '2018-11', '2018-12', '2019-1',  '2019-2',
  '2019-3',  '2019-4',  '2019-5',  '2019-6',  '2019-7',
  '2019-8',  '2019-9',  '2019-10', '2019-11', '2019-12',
  '2020-1',  '2020-2',  '2020-3',  '2020-4',  '2020-5',
  '2020-6',  '2020-7',  '2020-8',  '2020-9',  '2020-10',
  '2020-11', '2020-12', '2021-1',  '2021-2',  '2021-3',
  '2021-4',  '2021-5',  '2021-6',  '2021-7',  '2021-8',
  '2021-9',  '2021-10', '2021-11', '2021-12'
]

In [ ]:
Promise.all(yms.map(ym => {
    var q = `
            select count(CASE when repo_id = '22542579' and type = 'IssuesEvent' and action = 'opened' and toYear(created_at) = ${ym['y']} and toMonth(created_at) = ${ym['m']} THEN 1 ELSE NULL END) as createIssue,
            count(CASE when repo_id = '22542579' and type = 'IssuesEvent' and action = 'closed' and toYear(created_at) = ${ym['y']} and toMonth(created_at) = ${ym['m']} THEN 1 ELSE NULL END) as closeIssue
            from github_log.events
            `
    return query(q);
})).then(data => {
    var createIssueCount = [], closeIssueCount = [],tableDict = [];
    data.forEach((d, i) => {
        createIssueCount.push(d[0].createIssue);
        closeIssueCount.push(d[0].closeIssue);
        tableDict.push({'ym': yms_axis[i], 'createIssue': createIssueCount[createIssueCount.length-1], 'closeIssue': closeIssueCount[closeIssueCount.length-1]});
    });
    openDigger.render.plotly([
        {x: yms_axis, y: createIssueCount, mode: 'scatter', name: '每月打开issue数量'},
        {x: yms_axis, y: closeIssueCount, mode: 'scatter', name: '每月关闭Issue数量'},
    ], {
        title: '每月打开 Issue 和 关闭 Issue 的个数',
    });
    console.table(tableDict.map((d) => {
        return {
            '日期': d.ym,
            '打开Issue数量': d.createIssue,
            '关闭Issue数量': d.closeIssue,
        };
    }));
})

In [20]:
query(
    `select avg(issue_days) as issue_avg,median(issue_days) as issue_mid
    from(
    select toRelativeDayNum(issue_closed_at)-toRelativeDayNum(issue_created_at) as issue_days 
    from github_log.events where repo_id='22542579' and type='IssuesEvent'
    )`
).then(res=>{    console.log(res);});

[ { issue_avg: null, issue_mid: null } ]
